In [1]:
import sys
import importlib

import random 
import networkx as nx

sys.path.append("../")

from src import utils
from src.utils import *

from src import plot_utils
from src import graph_utils
from src import inpaint_utils
from src import fiberatlas_utils

## Description
### Regressing Bundle Activities
Considering different edge combination model:
- Fully connected
- Randomized edge distribution

In [50]:
scale = 1
connFilename = f'../../atlas_data/fiber_atlas/probconnatlas/wm.connatlas.scale{scale}.h5'
hf = h5py.File(connFilename, 'r')

centers = np.array(hf.get('header').get('gmcoords'))
nsubject = hf.get('header').get('nsubjects')[()]
dim = hf.get('header').get('dim')[()]
fiber_affine = hf.get('header').get('affine')[()]

gmregions_names = hf.get('header').get('gmregions')[()]
nb_regions = gmregions_names.shape[0]

gm_mask_subj = nib.load('../../atlas_data/moviedata_fMRI_eg/gm_mask_subj7.nii').get_fdata() 
wm_mask_subj = (gm_mask_subj + 1) % 2

consistency_view = fiberatlas_utils.get_aggprop(hf, 'consistency')
length_view = fiberatlas_utils.get_aggprop(hf, 'length')
nbStlines_view = fiberatlas_utils.get_aggprop(hf, 'numbStlines')
nb_regions = consistency_view.shape[0]

In [64]:
thresh_subjapp = int(np.ceil(nsubject * 0.1)) 

X_original = []
bundles_labels = []
for i in tqdm(range(1,nb_regions + 1)):
    for j in range(i,nb_regions + 1):
        tmp = fiberatlas_utils.get_bundles_betweenreg(hf, i, j, verbose=False)
        if tmp is None: continue
        if np.sum(tmp[:,3] >= (thresh_subjapp)) == 0: continue
        bundles_labels.append((i,j))
        vec = np.zeros(nb_regions)
        vec[i-1] = 1.0
        vec[j-1] = 1.0
        X_original.append(vec)

X_original = np.array(X_original)

100%|██████████| 95/95 [00:21<00:00,  4.37it/s]


In [74]:
X_full = []
bundles_labels_full = []
for i in tqdm(range(1,nb_regions + 1)):
    for j in range(i,nb_regions + 1):
        bundles_labels_full.append((i,j))
        vec = np.zeros(nb_regions)
        vec[i-1] = 1.0
        vec[j-1] = 1.0
        X_full.append(vec)

X_full = np.array(X_full)

100%|██████████| 95/95 [00:00<00:00, 31229.63it/s]


In [83]:
region_ftimecourse = load(f"../../atlas_data/moviedata_fMRI_eg/yasseratlased_fmri/ftimecourse_95_scale{scale}.pkl")
regions_in_voxels = load(f'../../atlas_data/fiber_atlas/regions95_voxels_scale{scale}.pkl')[:,:,:,1:]
nb_time = region_ftimecourse.shape[1]

In [91]:
p1, p2 = 0, 0

dummy_space_reg = np.zeros((len(X_original), len(X_original)))
dummy_temp_reg = np.zeros((nb_time, nb_time))
Xmult = np.array([X_original.T for _ in range(region_ftimecourse.shape[-1])])
bundle_opt_original, logs = inpaint_utils.optimize_lreg(Xmult, region_ftimecourse, Ls=dummy_space_reg, Lt=dummy_temp_reg,
                                               verbose=True, num_epochs=200, logging=True, p1=p1, p2=p2, lr=1)

dummy_space_reg = np.zeros((len(X_randomized_deg), len(X_randomized_deg)))
Xmult = np.array([X_randomized_deg.T for _ in range(region_ftimecourse.shape[-1])])
bundle_opt_rand, logs = inpaint_utils.optimize_lreg(Xmult, region_ftimecourse, Ls=dummy_space_reg, Lt=dummy_temp_reg, 
                                               verbose=True, num_epochs=200, logging=True, p1=p1, p2=p2, lr=1)

dummy_space_reg = np.zeros((len(X_full), len(X_full)))
Xmult = np.array([X_full.T for _ in range(region_ftimecourse.shape[-1])])
bundle_opt_full, logs = inpaint_utils.optimize_lreg(Xmult, region_ftimecourse, Ls=dummy_space_reg, Lt=dummy_temp_reg, 
                                               verbose=True, num_epochs=200, logging=True, p1=p1, p2=p2, lr=1)

100%|██████████| 200/200 [04:50<00:00,  1.45s/it]


Losses are decomposed into:
generic loss=tensor([55.7381])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([55.7381])


100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


Losses are decomposed into:
generic loss=tensor([29.3055])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([29.3055])


100%|██████████| 200/200 [11:45<00:00,  3.53s/it]

Losses are decomposed into:
generic loss=tensor([84.4159])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([84.4159])


In [104]:
utils.save('../resources/null_bundle_activities/bundle_opt_full.pkl', bundle_opt_full)
utils.save('../resources/null_bundle_activities/bundle_opt_original.pkl', bundle_opt_original)

In [105]:
conn_mat = np.zeros_like(consistency_view)
bundles_labels_reordered = np.array(bundles_labels) - 1
for pair in bundles_labels_reordered:
    conn_mat[pair[0], pair[1]] = 1.
    conn_mat[pair[1], pair[0]] = 1.
z = conn_mat.sum(axis=0)

random_trial = {}
for k in range(20):
    G = nx.configuration_model(z, seed=k)
    randomized_adj = nx.adjacency_matrix(G).todense()
        
    X_randomized_deg = []
    bundles_labels_rand = []
    for i in tqdm(range(1,nb_regions + 1)):
        for j in range(i,nb_regions + 1):
            if randomized_adj[i-1,j-1]:
                bundles_labels_rand.append((i,j))
                vec = np.zeros(nb_regions)
                vec[i-1] = 1.0
                vec[j-1] = 1.0
                X_randomized_deg.append(vec)

    X_randomized_deg = np.array(X_randomized_deg)

    dummy_space_reg = np.zeros((len(X_randomized_deg), len(X_randomized_deg)))
    Xmult = np.array([X_randomized_deg.T for _ in range(region_ftimecourse.shape[-1])])
    bundle_opt_rand, logs = inpaint_utils.optimize_lreg(Xmult, region_ftimecourse, Ls=dummy_space_reg, Lt=dummy_temp_reg, 
                                                verbose=True, num_epochs=200, logging=True, p1=p1, p2=p2, lr=1)

    random_trial[k] = bundle_opt_rand

100%|██████████| 200/200 [03:13<00:00,  1.03it/s]


Losses are decomposed into:
generic loss=tensor([15.6205])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([15.6205])


100%|██████████| 200/200 [02:56<00:00,  1.13it/s]


Losses are decomposed into:
generic loss=tensor([2.4244])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([2.4244])


100%|██████████| 200/200 [02:51<00:00,  1.17it/s]


...

Losses are decomposed into:
generic loss=tensor([14.0912])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([14.0912])


100%|██████████| 200/200 [02:51<00:00,  1.17it/s]


Losses are decomposed into:
generic loss=tensor([20.4410])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([20.4410])


100%|██████████| 200/200 [02:57<00:00,  1.13it/s]

Losses are decomposed into:
generic loss=tensor([42.5198])
spatialloss=tensor([0.])
temporalloss=tensor([0.])
sumloss=tensor([42.5198])


In [106]:
utils.save('../resources/null_bundle_activities/bundle_opt_rand_trials.pkl', random_trial)

In [ ]:
# NOTE: Close the opened h5 file
hf.close()